In [3]:
import sys
import os
import torch
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, '..'))
from model.model_config import ModelConfig
from data.data_config import DataConfig

model_config = ModelConfig("qwen25", "/home/wuyi/wy/NPR-RL/assets/model/qwen25_ins")
model, tokenizer = model_config.load_model()
data_config = DataConfig("/home/wuyi/wy/NPR-RL/assets/Train", model_config, is_test=True)
dataset = data_config.load_dataset()

Loading model......


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.52it/s]


Model loaded......
Loading data......
total number of data: 144641
train number of data: 10


100%|██████████| 10/10 [00:00<00:00, 12289.20it/s]


In [5]:
data = dataset[0]
print(data.keys())
print(model)


dict_keys(['idx', 'bug_method', 'bug_line', 'fix_line'])
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
 

In [ ]:

logits = model(input_ids, input_mask, target_ids, target_mask).logits
print(logits.shape)

In [ ]:
model = model.float()
output = model.generate(
        input_ids = data["input_ids"], 
        attention_mask = data["input_ids"].ne(tokenizer.pad_token_id),
        do_sample=False,
        top_p=1,
        top_k=10,
        temperature=1.5,
        max_length=32,
        return_dict_in_generate=True,
        output_scores=True
    )

In [ ]:
import torch
print(output.sequences.squeeze(0).shape)
print(torch.stack(output.scores, dim=0).squeeze(1).shape)
scores = torch.stack(output.scores, dim=0).squeeze(1)
print(scores)
logits = scores.softmax(dim=-1)
token = output.sequences.squeeze(0)[:-1]
token_logits = logits[torch.arange(token.shape[0]), token]
log_logits = torch.log(token_logits).sum().item()
print(log_logits)
print(torch.isnan(token_logits).any())
print(torch.isinf(token_logits).any())


output_txt = tokenizer.decode(
    token, skip_special_tokens=True
)
print(output_txt)

In [9]:
a = ["12313", "21313"]
a.append("asdad")
print(a)
a = "\n".join(a)
print(1)
print(typeof(a))

['12313', '21313', 'asdad']
1


NameError: name 'typeof' is not defined